# Lab 14: Groundwater monitoring with GRACE

**Purpose:** The following tutorial details how to use observations from the Gravity Recovery and Climate Experiment (GRACE) to evaluate changes in groundwater storage for a large river basin. Here, you will learn how to apply remote sensing estimates of total water storage anomalies, land surface model output, and in situ observations to resolve groundwater storage changes in California’s Central Valley. The following method has been applied to study water storage changes around the world and can be ported to quantify groundwater storage change for major river basins.  

In [ ]:
%pylab inline

In [ ]:
# import ee api and geemap package
import ee
import math
import geemap
import pandas as pd
from scipy import stats
from geemap import colormaps as cmaps

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Background

Since 2002, the Gravity Recovery and Climate Experiment (GRACE) and the follow-on mission, GRACE-FO, have provided a new vantage to track changes in water resources (Tapley et al. 2004). GRACE holds the unique ability to directly track changes in total water storage anomalies (TWSa), according to the following equation:

$TWSa = CANa + SMa + SWa + SWEa + GWa$

where $CANa$ is canopy water storage anomaly, $SMa$ is the soil moisture anomaly, $SWa$ is the surface water anomaly, $SWEa$ is the snow water equivalent anomaly, and $GWa$ is the groundwater storage anomaly. By incorporating supplemental observations from other remote sensing platforms and land surface models and rearranging the equation, scientists have been able to resolve changes in groundwater storage within major river basins around the planet (Famiglietti et al. 2014). From Bangladesh (Purdy et al. 2019) and India (Rodell et al. 2009) to the Middle East (Voss et al. 2013) and the American Southwest (Castle et al. 2014), declining groundwater storage changes have emerged with varying levels of severity (Richey et al. 2015). Like many other regions around the world, California shares the problem of groundwater overreliance (Famiglietti et al. 2011). This tutorial demonstrates  the analytical steps to resolve groundwater storage changes using GRACE for California’s Central Valley.

## Define AOI - Central Valley

Our area of interes is the Central Valley in California. This area is home to California's major mountain water source, the snowpack of the Sierra Nevada range. The Central Valley is the most productive agricultural region in the U. S., growing 8 percent of the food produced in the U. S. by value ([Faunt, 2009](https://pubs.usgs.gov/pp/1766/)). It accounts for 1/6 of the country's irrigated land and supplies 1/5 of the demand for groundwater in the United States. The area is the second most pumped aquifer in the U. S. after the High Plains aquifer making it an interesting study area for groundwater.

In [ ]:
# Import Basins
basins = ee.FeatureCollection("USGS/WBD/2017/HUC04");


# filter basins to the Central Valley
# Extract the 3 HUC 04 basins for the Central Valley
central_valley = basins.filter(
    ee.Filter.Or(
        ee.Filter.eq('huc4','1802'),
        ee.Filter.eq('huc4','1803'),
        ee.Filter.eq('huc4','1804')
    )
)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(central_valley, 6); 

Map.addLayer(central_valley,{},"Central Valley")

Map.addLayerControl()

Map

### Import GRACE Data and display Total Water Storage

GRACE can directly track changes in total water storage anomalies (TWSa). Changes in TWSa indicate which regions are gaining or losing water. Remember there are multiple GRACE solutions but we will use the Mascon with coastal resolution improvements (CRI) ([Wiese et al., 2016]( https://doi.org/10.1002/2016WR019344)).

We will import the GRACE collection and calculate the average change in TWSa for 2003-2016 as well as extract a time series for the Central Valley area of interest.

In [ ]:
# define start time
start_time = ee.Date("2003-01-01")
end_time = ee.Date("2017-01-01")

In [ ]:
grace = ee.ImageCollection("NASA/GRACE/MASS_GRIDS/MASCON_CRI").filterDate(start_time, end_time)

The GRACE data imported here have already been processed to provide units of total water storage anomalies. The data contained in this dataset are units of "equivalent water thickness" anomalies. However, we need to apply a gain (scale) factor to the data so that we can accurately compare to hydrologic trends ([Landerer & Swenson, 2012](https://doi.org/10.1029/2011WR011453)).

There is a file on Learning Suite for you to upload and use, otherwise load in the file shared.

In [ ]:
# load in the gain factor image
# change to the asset path that you uploaded
gain_factor = ee.Image("users/kmarkert/BYUCE594/CLM4_SCALE_FACTOR_JPL_MSCNv02CRI")

In [ ]:
# function to apply gain to each image
def apply_gain(img):
    return img.multiply(gain_factor).copyProperties(img,["system:time_start"])

# select the GRACE liquid water equivalent data and apply gain factor
twsa = grace.select('lwe_thickness').map(apply_gain)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(central_valley, 6); 

Map.addLayer(twsa.mean(),{"min":-10,"max":10,"palette":cmaps.get_palette("Spectral")}, "Total Water Storage anomaly", opacity=0.75)
Map.addLayer(central_valley,{},"Central Valley")

Map.add_colorbar({"min":-10,"max":10,"palette":cmaps.get_palette("Spectral")}, label="LWE [cm]")

Map.addLayerControl()

Map

In [ ]:
# define a function that will calculate per-band average values for Central Valley
# and set the averages to the image
def calc_timeseries(img):
    val = img.reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry = central_valley.geometry(1e4),
        scale = img.select([0]).projection().nominalScale()
    )

    return img.set(val)

In [ ]:
# get the time series to total water storage
twsa_timeseries = twsa.map(calc_timeseries)

In [ ]:
# extract out the timeseries information from the collection
timeseries = twsa_timeseries.aggregate_array("lwe_thickness").getInfo()
timestamp = twsa_timeseries.aggregate_array("system:time_start").getInfo()

In [ ]:
# convert the data into a pandas DataFrame
dates = pd.to_datetime(np.array(timestamp)*1e6)
twsa_series = pd.Series(timeseries,index=dates,name="TWSa")

In [ ]:
ax = twsa_series.plot(figsize=(10,7));
ax.hlines(0, twsa_series.index[0], twsa_series.index[-1],ls='--',color="k")
ax.set_ylabel("TWS anomaly [cm]")
show()


### Estimate the Linear Trend in TWSa Over Time

GRACE data only calculates the anomalies (difference from mean) and does not measure total/absolute water storage. However, we can quantify when mass changes and by how much over a long time period and we will do that here by estimating the trend in time for each pixel.

In [ ]:
# define a function that adds the coefficients needed for regression
def add_variables(img):
    # Compute time in fractional years since the epoch.
    date = img.date()
    time = date.difference(start_time, 'year')
    const = ee.Image.constant(1)
    return img.addBands(ee.Image(time).float().rename("time")).addBands(const)


In [ ]:
# add the coefficients for regression to the images
twsa_time = twsa.map(add_variables)

In [ ]:
# List of the independent variable names
independents = ee.List(['constant', 'time'])

# Name of the dependent variable.
dependent = ee.String('lwe_thickness')


In [ ]:
# Compute a linear trend.  This will have two bands: 'residuals' and 
# a 2x1 band called coefficients (columns are for dependent variables).
twsa_trend = (
    twsa_time
    .select(independents.add(dependent))
    .reduce(ee.Reducer.linearRegression(independents.length(), 1))
)


The coefficients image is a two-band array image in which each pixel contains values for the slope and offset. We will need convert the array to an image:


In [ ]:
# Flatten the coefficients into a 2-band image
coefficients = (
    twsa_trend
    .select('coefficients')
    .arrayProject([0])
    .arrayFlatten([independents])
)

Next, you can visualize the GRACE trends to capture the spatial scales on which GRACE can resolve TWSa. GRACE is adept at capturing these changes *only for larger basins*.


In [ ]:
# Create a layer of the TWSa slope to add to the map
slope = coefficients.select('time');

In [ ]:

# Visualize the results
Map = geemap.Map()

Map.centerObject(central_valley, 6); 

Map.addLayer(slope.clip(central_valley), {"min": -3.5, "max": 3.5, "palette":cmaps.get_palette("bwr_r")}, 'TWSa Trend', opacity=0.75);
Map.addLayer(central_valley,{},"Central Valley",opacity=0.2)

Map.addLayerControl()

Map

The slope layer reveals that the southern basins (Tulare and San Joaquin) experiences the largest negative changes in TWSa over the time period.


### Calculating surface storage changes

As mentioned, GRACE calcualtes the total water storage anomalies. Therefore, to extract out the groundwater signal, we will need to remove the signal from other storage changes. This is typically done by calculating the surface storage anomalies from the Global Land Data Assimilation System (GLDAS) data.

GLDAS comes at 3hr temporal resolution and it the absolute values (e.g. mm) of storage so to compare with GRACE we will need to perform some temporal averaging and calculate the anomalies.

In [ ]:
# get the grace image time windows 
ts = grace.aggregate_array("system:time_start")
te = grace.aggregate_array("system:time_end")

# calculate how many images there are and get a list to iterate over
n = ts.length()
iter = ee.List.sequence(0,n.subtract(1))

In [ ]:
# define the bands from GLDAS that we want to use 
swe_bands = ee.List(["SWE_inst"])
canopy_bands = ee.List(["CanopInt_inst"])
soil_bands = ee.List(["SoilMoi0_10cm_inst","SoilMoi10_40cm_inst","SoilMoi40_100cm_inst","SoilMoi100_200cm_inst"])

bands_select = swe_bands.cat(canopy_bands).cat(soil_bands)

In [ ]:
# load in the GLDAS collection
# filter by time
# select only the surface storage bands of interes
gldas = (
    ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")
    .filterDate(start_time, end_time)
    .select(bands_select)
)

In [ ]:
# function to calculate surface storages at GRACE time periods
def calc_surface_storage(i):
    # get the time window from GRACE time info
    t1 = ee.Date(ts.get(i))
    t2 = ee.Date(te.get(i))

    # filter by time window and calculate average
    storages = gldas.filterDate(t1,t2).mean()
    # extract out specific components and rename
    # we will sum for the soil column though
    soil_storage = storages.select(soil_bands).reduce("sum").rename("soil")
    snow_storage = storages.select(swe_bands).rename("snow")
    canopy_storage = storages.select(canopy_bands).rename("canopy")

    # kg/m^2 to cm
    conversion = 0.1

    # combine the surface storage components and set time info
    return (
        ee.Image.cat([
            soil_storage, 
            snow_storage, 
            canopy_storage
        ])
        .multiply(conversion)
        .set("system:time_start", t1.millis(), "system:time_end", t2.millis())
    )
   
# apply calculation of monthly surface storages
surface_storages = ee.ImageCollection.fromImages(iter.map(calc_surface_storage))

To calculate anomalies (i.e. difference from mean) we need to calculate the mean storage for a baseline (or reference) period. This period should align with the GRACE baseline period so we are comparing the anomalies from the same time period. The [GRACE page](https://grace.jpl.nasa.gov/data/get-data/jpl_global_mascons/) it specifies the baseline as 2004-2009 so we will use the baseline this baseline. However, this baseline can easily change if needed.

In [ ]:
# set baseline start and end time to calculate anomalies
baseline_start = ee.Date("2004-01-01")
baseline_end = ee.Date("2010-01-01")

In [ ]:
# calculate average values for the storage components
storage_averages = surface_storages.filterDate(baseline_start, baseline_end).mean()

Now we can calculate the anomalies by simply subtracting the mean values from the surface storage components:

In [ ]:
# function to remove baseline mean
def calc_anomaly(img):
    anomaly = img.subtract(storage_averages)
    return anomaly.copyProperties(img, ["system:time_start"])

# apply function to calculate anomalies
surface_anomalies = surface_storages.map(calc_anomaly)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(central_valley, 6); 

Map.addLayer(surface_anomalies.mean(), {"min": -1.5, "max": 1.5}, 'SSa Trend', opacity=0.75);
Map.addLayer(central_valley,{},"Central Valley",opacity=0.2)

Map.addLayerControl()

Map

To confirm we have our time series of anomalies, we will plot the series for Central Valley.

In [ ]:
# apply our time series function for Central Valley
surface_anomlay_ts = surface_anomalies.map(calc_timeseries)

In [ ]:
# extract out the timeseries information from the collection
snow_ts = surface_anomlay_ts.aggregate_array("snow").getInfo()
soil_ts = surface_anomlay_ts.aggregate_array("soil").getInfo()
canopy_ts = surface_anomlay_ts.aggregate_array("canopy").getInfo()
timestamp = surface_anomlay_ts.aggregate_array("system:time_start").getInfo()

In [ ]:
# convert the data into a pandas DataFrame
dates = pd.to_datetime(np.array(timestamp)*1e6)
surface_series = pd.DataFrame({"soil_anomaly": soil_ts, "snow_anomaly": snow_ts, "canopy_anomaly":canopy_ts},index=dates,)

In [ ]:
# plot the timeseries
axs = surface_series.plot(figsize=(10,7),subplots=True);
for ax in axs:
    ax.hlines(0, surface_series.index[0], surface_series.index[-1],ls='--',color="k")
    ax.set_ylabel("Storage anomaly [cm]")
show()


### Extracting out signals of GroundWater changes

Now that we have the anomalies for the different surface water components, we remove that signal from the total water storage anomalies to get the groundwater storage anomalies. To do this efficiently, we will join the two collections based on time. Since we used the same time ranges to calculate the surface storage anomalies, the time should line up nicely. 

To apply the join, we will create a filter based on time. We will set this to a ten day window just to be safe and make sure we join the data we need correctly.

In [ ]:
# Define an allowable time difference: ten days in milliseconds.
ten_day_millis = 24 * 60 * 60 * 1000 * 10

# Create a time filter to define a match as overlapping timestamps.
time_filter = ee.Filter.Or(
    # use max difference filter to specify only one day difference
    # checks one day on either side of observation
    ee.Filter.maxDifference(
        difference= ten_day_millis,
        leftField= 'system:time_start',
        rightField= 'system:time_start'
    )
);

Now that we have a filter, we need to define our join. This specifies what the results property name will be (in this case `surface_storages`) so we know what to look for later.

In [ ]:
# Define the join.
# this is "saveBest" which will give us the image closest in time to what we want
storages_join = ee.Join.saveBest(
  matchKey= 'surface_storages', # this will be the name of the result in the collection
  measureKey= 'timeDiff'
)

In [ ]:
# Apply the join.
# uses soil_moisture as the collection to join to and applies filter on surface reflectance data
joined_storages = ee.ImageCollection(storages_join.apply(twsa, surface_anomalies, time_filter))

Lastly, let's compute groundwater water storage changes:

In [ ]:
# function to calculate ground water storage anomalies
# by subtracting the surface water anomalies
def extract_groundwater(img):
    surface_anomalies = ee.Image(img.get("surface_storages")).reduce("sum")
    return img.resample().subtract(surface_anomalies).copyProperties(img, ["system:time_start"])

# apply the calculation for groundwater storage
gwsa = joined_storages.map(extract_groundwater)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(central_valley, 6); 

Map.addLayer(gwsa.mean(), {"min": -10, "max": 10, "palette":cmaps.get_palette("Spectral")}, 'GWSa', opacity=0.75);
Map.addLayer(central_valley,{},"Central Valley",opacity=0.2)

Map.add_colorbar({"min": -10, "max": 10, "palette":cmaps.get_palette("Spectral")}, label="Groundwater anomaly [cm]")

Map.addLayerControl()

Map

### Groundwater trends

Now that we have a collection of the groundwater storage anomalies, we can calculate a time series for the region of interest. We will focus our analysis on a drought period 2006-2011. This is a similar analysis to [Famiglietti et al.(2011)]( https://doi.org/10.1029/2010GL046442) so we can compare results.

In [ ]:
# define drought start/end dates
drought_start = ee.Date("2006-04-01")
drought_end = ee.Date("2010-04-01")

In [ ]:
# filter dates and add time information
gwsa_time = gwsa.filterDate(drought_start,drought_end).map(add_variables)

In [ ]:
# Compute a linear trend.  This will have two bands: 'residuals' and 
# a 2x1 band called coefficients (columns are for dependent variables).
gwsa_trend = (
    twsa_time
    .select(independents.add(dependent))
    .reduce(ee.Reducer.linearRegression(independents.length(), 1))
)


In [ ]:
# Flatten the coefficients into a 2-band image
coefficients = (
    gwsa_trend
    .select('coefficients')
    .arrayProject([0])
    .arrayFlatten([independents])
)

In [ ]:
# Create a layer of the TWSa slope to add to the map
slope = coefficients.select('time');

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(central_valley, 6); 

Map.addLayer(slope.clip(central_valley), {"min": -3.5, "max": 3.5, "palette":cmaps.get_palette("bwr_r")}, 'GWSa Trend', opacity=0.75);
Map.addLayer(central_valley,{},"Central Valley",opacity=0.2)

Map.addLayerControl()

Map

In [ ]:
cv_slope = slope.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = central_valley.geometry(1e4),
    scale=slope.projection().nominalScale()
)

In [ ]:
rate = cv_slope.get("time").getInfo()

print(f"Average Central Valley GWSa rate of change: {rate:.4f} cm/yr")

In [ ]:
# apply time series calculation over Central Valley
gwsa_timeseries = gwsa_time.map(calc_timeseries).filter(ee.Filter.neq("lwe_thickness",None))

In [ ]:
# extract out the timeseries information from the collection
timeseries = gwsa_timeseries.aggregate_array("lwe_thickness").getInfo()
year = gwsa_timeseries.aggregate_array("time").getInfo()
timestamp = gwsa_timeseries.aggregate_array("system:time_start").getInfo()

In [ ]:
# convert the data into a pandas DataFrame
dates = pd.to_datetime(np.array(timestamp)*1e6)
gwsa_drought_df = pd.DataFrame({"GWSa": timeseries, "year": year},index=dates)

In [ ]:
regression = stats.linregress(gwsa_drought_df["year"], gwsa_drought_df["GWSa"])

gwsa_drought_df["trend"] = gwsa_drought_df["year"] * regression[0] + regression[1]

In [ ]:
ax = gwsa_drought_df[["GWSa","trend"]].plot(figsize=(10,7));
ax.hlines(0, gwsa_drought_df.index[0], gwsa_drought_df.index[-1],ls='--',color="k")
ax.set_ylabel("GWS anomaly [cm]")
show()

In [ ]:
print(f"Calculated Central Valley GWSa rate of change: {regression[0]:.4f} cm/yr")

Based on the calculated rates of change, there was is a high reliance on groundwater. Another paper, [Famiglietti et al., 2011]( https://doi.org/10.1029/2010GL046442) calculated the change as  -3.89 ± 0.95 cm/yr, whereas our estimates were lower when caculating the slope then averaging and higher when averaging then calculating slope. However, in their paper they only used the Sacramento and San Joaquin River Basins (top two basins) whereas we used all three basins covering Central Valley and applied the gain factors for comparing to hydrologic data. 

Funny enough, if you average the two rates of change calculated then you get the same answer as Famiglietti et al., 2011....